# Set Up Environment

In [1]:
# cell 1
# initialize all packages used for deployment of OMM SCOREs
# use 
# pip install -r _deployment/requirement.txt
# to install all dependencies

from iconservice import AddressPrefix, Address
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder,DepositTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from collections import defaultdict
import requests
import random
import time

import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass
from dotenv import dotenv_values

deployment_status = {}
print(1)

1


In [2]:
# cell 2
config = dotenv_values("./_deployment/.env.jupyter")

EXA = 10 ** 18
ICX = 10 ** 18

NETWORK = "sejong"

PREPS_LIST_FILE = config["PREPS_LIST_FILE"]
with open(PREPS_LIST_FILE, "r") as file:
    PREP_LIST = json.load(file)

TIMESTAMP = int(config["TIMESTAMP"])
CONTRACT_ADDRESSES_FILE = config["CONTRACT_ADDRESS_FILE"]

GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
WORKER_WALLET_ADDRESS=config["WORKER_WALLET_ADDRESS"]


#lending and borrow reward
LENDING_BORROW_PERCENTAGE = int(config["LENDING_BORROW_PERCENTAGE"]) * EXA // 100
#LP and OMM staking reward
LP_OMM_STAKING_PERCENTAGE = int(config["LP_OMM_STAKING_PERCENTAGE"]) * EXA // 100
#DAO reward percentage
DAO_DIST_PERCENTAGE = int(config["DAO_DIST_PERCENTAGE"]) * EXA // 100
#Worker reward 
WORKER_DIST_PERCENTAGE = int(config["WORKER_DIST_PERCENTAGE"]) * EXA // 100

assert (LENDING_BORROW_PERCENTAGE + LP_OMM_STAKING_PERCENTAGE + DAO_DIST_PERCENTAGE + WORKER_DIST_PERCENTAGE) // EXA == 1

ICX_EMISSION = int(config["ICX_PERCENTAGE"])
OICX_EMISSION = int(config["OICX_EMISSION"]) * ICX_EMISSION * EXA // 10000
DICX_EMISSION = int(config["DICX_EMISSION"]) * ICX_EMISSION * EXA // 10000

USDS_EMISSION = int(config["USDS_PERCENTAGE"])
OUSDS_EMISSION = int(config["OUSDS_EMISSION"]) * USDS_EMISSION * EXA // 10000
DUSDS_EMISSION = int(config["DUSDS_EMISSION"]) * USDS_EMISSION * EXA // 10000

IUSDC_EMISSION = int(config["IUSDC_PERCENTAGE"])
OIUSDC_EMISSION = int(config["OIUSDC_EMISSION"]) * IUSDC_EMISSION * EXA // 10000
DIUSDC_EMISSION = int(config["DIUSDC_EMISSION"]) * IUSDC_EMISSION * EXA // 10000

bnUSD_EMISSION = int(config["bnUSD_PERCENTAGE"])
ObnUSD_EMISSION = int(config["ObnUSD_EMISSION"]) * bnUSD_EMISSION * EXA // 10000
DbnUSD_EMISSION = int(config["DbnUSD_EMISSION"]) * bnUSD_EMISSION * EXA // 10000

BALN_EMISSION = int(config["BALN_PERCENTAGE"])
OBALN_EMISSION = int(config["OBALN_EMISSION"]) * BALN_EMISSION * EXA // 10000
DBALN_EMISSION = int(config["DBALN_EMISSION"]) * BALN_EMISSION * EXA // 10000


OMM_EMISSION = int(config["OMM_PERCENTAGE"])
OOMM_EMISSION = int(config["OOMM_EMISSION"]) * OMM_EMISSION * EXA // 10000
DOMM_EMISSION = int(config["DOMM_EMISSION"]) * OMM_EMISSION * EXA // 10000

assert (OICX_EMISSION + DICX_EMISSION + OUSDS_EMISSION + DUSDS_EMISSION + \
        OIUSDC_EMISSION + DIUSDC_EMISSION + ObnUSD_EMISSION + DbnUSD_EMISSION +\
        OBALN_EMISSION + DBALN_EMISSION + OOMM_EMISSION + DOMM_EMISSION) // EXA == 1

#LP and OMM staking reward

OMM_SICX_DIST_PERCENTAGE = int(config["OMM_SICX_DIST_PERCENTAGE"]) * EXA // 100
OMM_USDS_DIST_PERCENTAGE = int(config["OMM_USDS_DIST_PERCENTAGE"]) * EXA // 100
OMM_USDC_DIST_PERCENTAGE = int(config["OMM_USDC_DIST_PERCENTAGE"]) * EXA // 100
OMM_DIST_PERCENTAGE = int(config["OMM_DIST_PERCENTAGE"]) * EXA // 100

assert (OMM_SICX_DIST_PERCENTAGE + OMM_USDS_DIST_PERCENTAGE + OMM_USDC_DIST_PERCENTAGE + OMM_DIST_PERCENTAGE) // EXA == 1

DEPOSIT_ICX_AMOUNT = int(config["DEPOSIT_ICX_AMOUNT"])
FEE_SHARING_TX_LIMIT = int(config["FEE_SHARING_TX_LIMIT"])
LOAN_ORIGINATION_PERCENTAGE = int(float(config["LOAN_ORIGINATION_PERCENTAGE"]) * EXA)
MINIMUM_OMM_STAKE = int(config["MINIMUM_OMM_STAKE"]) * EXA
OMM_UNSTAKING_PERIOD = int(config["OMM_UNSTAKING_PERIOD"])


DEPLOYER_WALLET_KEYSTORE = config["DEPLOYER_WALLET_KEYSTORE"]
POOL_CONFIGURATION_WALLET_KEYSTORE = config["POOL_CONFIGURATION_WALLET_KEYSTORE"]
CONTRACT_ADDRESS_FILE = config["CONTRACT_ADDRESS_FILE"]
AIRDROP_HOT_WALLET_ADDRESS=config["AIRDROP_HOT_WALLET_ADDRESS"]

BORROW_THRESHOLD=int(config["BORROW_THRESHOLD"])*EXA//100

_contract_file_loaded = False
print(1)

1


In [3]:
# cell 3
connections = {
    "mainnet": {"iconservice": "https://ctz.solidwallet.io", "nid": 1},
    "yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
    "euljiro": {"iconservice": "https://test-ctz.solidwallet.io", "nid": 2},
    "pagoda": {"iconservice": "https://zicon.net.solidwallet.io", "nid": 80},
    "goloop": {"iconservice": "http://18.237.205.52:9082/", "nid": 3},
    "local": {"iconservice": "http://localhost:9000/", "nid": 3},
    "berlin": {"iconservice": "https://berlin.net.solidwallet.io", "nid": 7},
    "lisbon": {"iconservice": "https://lisbon.net.solidwallet.io", "nid": 2},
    "sejong": {"iconservice": "https://sejong.net.solidwallet.io", "nid": 83}
}

env = connections[NETWORK]

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]


@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result


print(f"network -> {NETWORK}")
print(f"nid -> {NID}")

network -> sejong
nid -> 83


In [33]:
# cell 4
# initialized all methods
def get_status_and_print(to, method, response, logs = False) -> int:
    """
    get status and print response
    :param to:
    :param method:
    :param response:
    :return: status
    """
    status = response['status']
    print(f'SCORE {to} ::: Method {method} ::: Status: {status}')
    if logs:
        if len(response["eventLogs"]) > 0:
            for item in response["eventLogs"]:
                print(f'{item} \n')
        if status == 0:
            print(f'Failure: {response["failure"]}')
    return status


def _deploy_contract(_contract_name, params):
    deploy_transaction = DeployTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(GOVERNANCE_ADDRESS) \
        .nid(NID) \
        .nonce(100) \
        .step_limit(100000000000) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(_contract_name)) \
        .params(params) \
        .build()
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    pprint(tx_hash)
    res = get_tx_result(tx_hash)
    status = get_status_and_print(_contract_name, "deploy", res)

    return {
        "address": res.get('scoreAddress', ''),
        "status": status
    }


def deploy_contract(_contract_name):
    """
    deploy all contracts excepts oToken and dToken
    :param _contract_name:  name of contract - should match with package name
    :return: SCORE address
    """
    params = {}
    if _contract_name == "workerToken":
        params = {'_initialSupply': int(config.get("worker_token_initial")), '_decimals': 18}
    if _contract_name == "rewardDistribution":
        DISTRIBUTION_PERCENTAGE = [
            {"recipient": "worker", "percentage": f'{WORKER_DIST_PERCENTAGE}'},
            {"recipient": "daoFund", "percentage": f'{DAO_DIST_PERCENTAGE}'},
            {"recipient": "lendingBorrow", "percentage": f'{LENDING_BORROW_PERCENTAGE}'},
            {"recipient": "liquidityProvider", "percentage": f'{LP_OMM_STAKING_PERCENTAGE}'}
        ]
        params = {"_distPercentage": DISTRIBUTION_PERCENTAGE, "_startTimestamp": TIMESTAMP}

    if _contract_name not in ['addressProvider', "workerToken"]:
        params['_addressProvider'] = contracts['addressProvider']

    result = _deploy_contract(_contract_name, params)
    deployment_status[f"deploy_contract_{_contract_name}"] = result["status"]
    return result["address"]

def deposit_transaction(_from, _to, _value, _method="add"):
    """
    Deposit fee to contracts for fee sharing
    :param _from: transaction wallet
    :param _to: contract to deposit fee for fee sharing
    :param _value: amount to deposit
    :param _method: method
    :return: status of transaction
    """
    print(f"Depositing {_value} to {_to} contract for fee sharing")
    transaction = DepositTransactionBuilder() \
        .from_(_from.get_address()) \
        .to(contracts[_to]) \
        .nid(NID) \
        .value(_value) \
        .step_limit(1000000000) \
        .nonce(100) \
        .action(_method) \
        .build()
    
    signed_transaction = SignedTransaction(transaction, _from)
    tx_hash = icon_service.send_transaction(signed_transaction)
    response = get_tx_result(tx_hash)
    status = get_status_and_print(_to, _method, response)
    return status

def deploy_token_contract(token, params):
    """
    deploy  oTokens and dTokens
    :param token: token name
    :param params: parameter
    :return: SCORE address
    """
    name = params["name"]
    symbol = params["symbol"]
    decimals = params["decimals"]
    print(f"deploying {name}::{symbol}::{decimals}")
    params = {"_name": name, "_symbol": symbol, "_decimals": decimals, '_addressProvider': contracts['addressProvider']}

    result = _deploy_contract(token, params)
    deployment_status[f"deploy_{token}_contract_{symbol}"] = result["status"]
    return result["address"]


def send_tx(_to, _method, _params, _value, _wallet, _tx_detail = False) -> int:
    """
    send transaction to _to
    :param _to: destination conract name
    :param _method: method
    :param _params:  parameters
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    print(f'Calling {_method}, with parameters {_params} on the {_to} contract.')
    transaction = CallTransactionBuilder() \
        .from_(_wallet.get_address()) \
        .to(contracts[_to]) \
        .value(_value) \
        .step_limit(1_00_000_000) \
        .nid(NID) \
        .nonce(100) \
        .method(_method) \
        .params(_params) \
        .build()
    signed_transaction = SignedTransaction(transaction, _wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(f"{_to} ::> {_method} ::> {tx_hash}")
    response = get_tx_result(tx_hash)
    if _tx_detail:
        pprint(response)
    status = get_status_and_print(_to, _method, response)
    return tx_hash

def hexToInt(value):
    if type(value) is str:
        return int(value,0)
    if type(value) is dict:
        res ={}
        for k,v in value.items():
            if(not v.startswith("cx")):
                res[k]=int(v,0)
        return res

def call_tx(contact, method, params):
    call = CallBuilder()\
        .from_('hx91bf040426f226b3bfcd2f0b5967bbb0320525ce')\
        .to(contracts[contact])\
        .method(method)\
        .params(params)\
        .build()
    response = icon_service.call(call)
    return response

def send_icx(_to, _value, _wallet) -> int:
    """
    send transaction to _to
    :param _to: destination address
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    transaction = TransactionBuilder()\
        .from_(_wallet.get_address())\
        .to(_to)\
        .value(_value)\
        .step_limit(1_000_000_000)\
        .nid(NID)\
        .nonce(100)\
        .build()
    signed_transaction = SignedTransaction(transaction, _wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(f"{_to} ::> icx ::> {tx_hash}")
    get_tx_result(tx_hash)

@retry(Exception, tries=10, delay=1, back_off=2)
def ts(hash):
    _hash = str(hash)
    data = requests.get(f'https://sejong.tracker.solidwallet.io/v3/transaction/txDetail?txHash={_hash}')
    data = data.json()
    ts = data.get('data').get('createDate')[:-9]
    ts = datetime.fromisoformat(ts)
    return (int(datetime.timestamp(ts)) + 5 * 3600 + 45 * 60)

### Import deployer wallet

In [5]:
# cell 5
import getpass

worker_wallet = KeyWallet.load(bytes.fromhex('<pk>'))
deployer_wallet = KeyWallet.load(bytes.fromhex('<pk>'))
print(deployer_wallet.get_address())
print(icon_service.get_balance(deployer_wallet.get_address())/10**18)

hx5de49b9c9833f31d6bd237a074f86a30cc77126b
5667.4294888


In [10]:
with open(CONTRACT_ADDRESSES_FILE, 'rb') as f:
    contracts = pkl.load(f)

In [59]:
contracts['bOMM'] = "cxe0359335aaaa625c72d8429805cbf38f0258ee7a"
contracts["rewardWeightController"] = "cxc79754e827b7dc0a12b58092eae5c5d5ad6f8ce4"

In [45]:
pool_ids = {'OMM/USDS': '0x2a', 'OMM/IUSDC': '0x2b', 'OMM/sICX': '0x2c'}

### Generate Some Wallets To Test

In [78]:
private_keys = []
addresses = []
for i in range(4):
    wallet = KeyWallet.create()
    private_keys.append(wallet.get_private_key())
    addr = wallet.get_address()
    addresses.append(addr)    
    send_icx(addr, 10 * 10 ** 18,deployer_wallet)

hx687633a4c5870347e22fe0e3e07205b3b04485ad ::> icx ::> 0x0b1e71b619f474dcf8086972a838914b129ebecfde7cadc1bb12f1fdc8dbb62b
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
hx948679df839205f7e4c7bf261b92258c04f4653c ::> icx ::> 0x4d0bd94a5d117db5eeca64c92f9c8be952d14cd3e2b97162245db65ae7209a08
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
hx7a00e7b8d56b8391b753990ec4a9d6033aa4f6e5 ::> icx ::> 0xb495ed0fdb18df5c939b8fba1b06bf9b0dd593b6053095738bd14af8fc134d2f
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
hxdcb9443d393c67b357607d2e14248eeaad2938c3 ::> icx ::> 0x3aebb32b76a5ebf3494eb28bb17767399bd791e63d847311e360a4ec72d0a2e6
{'code': -31003, 'message': 'Executing: Executing'}, Ret

In [89]:
addresses

['hx687633a4c5870347e22fe0e3e07205b3b04485ad',
 'hx948679df839205f7e4c7bf261b92258c04f4653c',
 'hx7a00e7b8d56b8391b753990ec4a9d6033aa4f6e5',
 'hxdcb9443d393c67b357607d2e14248eeaad2938c3']

In [63]:
def yrs(n):
    return int(time.time() * 10 ** 6) +  n * 365 * 86400 * 10 ** 6

In [80]:
# Transaction Helper Functions
def icx_deposit(wallet, amount = 2 * 10 ** 18):     
    params = {"_amount": amount}
    _hash = send_tx('lendingPool','deposit',params, params['_amount'], wallet)
    return _hash

def deposit(token, wallet, amount):
    _depositAmount =amount
    params = {'_to': wallet.get_address(), '_value': amount}
    send_tx(token, 'transfer', params, 0, deployer_wallet)
    
    depositData = {'method': 'deposit', 'params': {'amount': _depositAmount}}
    data = json.dumps(depositData).encode('utf-8')

    params = {"_to": contracts['lendingPool'],
            "_value": _depositAmount, 
            "_data": data}

    _hash = send_tx(token,'transfer', params, 0, wallet)
    return _hash
    
def borrow(token, wallet, amount):
    params = {"_reserve": contracts[token],"_amount": amount}
    _hash = send_tx('lendingPool','borrow', params, 0, wallet)
    return _hash

def lock(wallet, amount: 20 * 10 ** 18, timePeriod = yrs(1)):
    params = {'_to': wallet.get_address(), '_value': amount}
    send_tx('ommToken', 'transfer', params, 0, worker_wallet)
    
    data = json.dumps({
        'method': 'createLock',
        'params': {
            'unlockTime': timePeriod
        }
    }).encode('utf-8')
    params = {
        '_to': contracts['bOMM'],
        '_value': amount,
        '_data': data
    }
    _hash = send_tx('ommToken','transfer',params,0,wallet)
    return _hash

def stake_lp(wallet, amt, tok):
    poolId = pool_ids[tok]
    _params = {
        '_to': wallet.get_address(),
        '_value': amt,
        '_id': poolId
    }
    print(f"transfer {tok} to user")    
    status = send_tx('dex', "transfer", _params, 0, worker_wallet)
    data = {
    'method': "stake"
    }

    data = json.dumps(data).encode('utf-8')
    
    _params = {
        '_to': contracts['stakedLp'],
        '_value': amt,
        '_id': poolId,
        '_data': data
    }
    print(f"transfer omm/{tok} to stakedLp")    
    _hash = send_tx('dex', "transfer", _params, 0, wallet)
    return _hash

In [ ]:
timestamps = defaultdict(dict)

In [91]:
#users perform transaction
for idx, pk in enumerate(private_keys):
    wallet = KeyWallet.load(bytes.fromhex(pk))
    addr = wallet.get_address()
    
    txHash = icx_deposit(wallet, random.randint(1,7) * 10 ** 18) 
    timestamps[addr]['oICX'] = ts(txHash)
    
    txHash = deposit('usds', wallet, 100*(idx+1) * 10 ** 18)
    timestamps[addr]['oUSDS'] = ts(txHash)
    
    txHash = borrow('iusdc', wallet, 20*(idx+1) * 10 ** 6)
    timestamps[addr]['dIUSDC'] = ts(txHash)
    
    txHash = lock(wallet, 200 * (idx+1)*10**18, yrs((idx+1) % 4))
    timestamps[addr]['bOMM'] = ts(txHash)
    
    txHash = stake_lp(wallet,(idx+1)*10**15, 'OMM/USDS')
    timestamps[addr]['OMM/USDS'] = ts(txHash)

Calling deposit, with parameters {'_amount': 5000000000000000000} on the lendingPool contract.
lendingPool ::> deposit ::> 0xff1bba8be11eed916973dd898eae06f5df1e348b7173a995c0659f51833b42f9
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE lendingPool ::: Method deposit ::: Status: 1
Calling transfer, with parameters {'_to': 'hx948679df839205f7e4c7bf261b92258c04f4653c', '_value': 100000000000000000000} on the usds contract.
usds ::> transfer ::> 0x7e256f3074b475d2252c5f83fb95906e3d4ee8363f750f59bd6a181586618277
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE usds ::: Method transfer ::: Status: 1
Calling transfer, with parameters {'_to': 'cx773b76e48c5959461bf60294e304d7d712d9a76c', '_value': 100000000000000000000, '_data': b'{"method": "deposit", "params": {"amount":

{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE lendingPool ::: Method deposit ::: Status: 1
'NoneType' object has no attribute 'get', Retrying in 1 seconds...
Calling transfer, with parameters {'_to': 'hxdcb9443d393c67b357607d2e14248eeaad2938c3', '_value': 300000000000000000000} on the usds contract.
usds ::> transfer ::> 0x36d0bb7f25738f3af0f025d589573f3bcaf70de57931697de30bc63a4a3ed93c
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE usds ::: Method transfer ::: Status: 1
Calling transfer, with parameters {'_to': 'cx773b76e48c5959461bf60294e304d7d712d9a76c', '_value': 300000000000000000000, '_data': b'{"method": "deposit", "params": {"amount": 300000000000000000000}}'} on the usds contract.
usds ::> transfer ::> 0x016c5a0d9e7005a50545a620f65468c5f364714db2ed74aef254b63d165c18ba
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 

### CHANGE ASSET AND TYPE WEIGHT

In [ ]:
"""
12:30 
# assetweight
oICX: 90%
oUSDS: 3%
dIUSDC: 1%

1:30
# assetWeight
oICX: 10%
oUSDS: 30%
dIUSDC: 40%

2:00
# assetWeight
oICX: 5%
oUSDS: 10%
dIUSDC: 80%

3:00
# assetWeight
oICX: 40%
oUSDS: 20%
dIUSDC: 35%
"""

In [87]:
# 12:30
#  params = {
#     'type': 'reserve',
#     'weights': [
#         {
#             'address': contracts['oICX'],
#             'weight': 90 * 10 ** 16          
#         },
#         {
#             'address': contracts['dICX'],
#             'weight': 0 * 10 ** 16          
#         },{
#             'address': contracts['oUSDS'],
#             'weight': 3 * 10 ** 16          
#         },{
#             'address': contracts['dUSDS'],
#             'weight': 0 * 10 ** 16          
#         },{
#             'address': contracts['oIUSDC'],
#             'weight': 0 * 10 ** 16          
#         },{
#             'address': contracts['dIUSDC'],
#             'weight': 1 * 10 ** 16          
#         },{
#             'address': contracts['obnUSD'],
#             'weight': 0         
#         },{
#             'address': contracts['dbnUSD'],
#             'weight': 0          
#         },{
#             'address': contracts['oOMM'],
#             'weight': 0         
#         },{
#             'address': contracts['dOMM'],
#             'weight': 0         
#         },{
#             'address': contracts['oBALN'],
#             'weight': 0          
#         },{
#             'address': contracts['dBALN'],
#             'weight': 0        
#         }
#     ],
#     'timestamp': 1653633900 * 10 ** 6
# }

# send_tx('rewardWeightController','setAssetWeight',params,0,deployer_wallet)


# 1:30
# params = {
#     'type': 'reserve',
#     'weights': [
#         {
#             'address': contracts['oICX'],
#             'weight': 10 * 10 ** 16          
#         },{
#             'address': contracts['oUSDS'],
#             'weight': 30 * 10 ** 16          
#         },{
#             'address': contracts['dIUSDC'],
#             'weight': 40 * 10 ** 16          
#         }
#     ],
#     'timestamp': 1653637500 * 10 ** 6
# }

# send_tx('rewardWeightController','setAssetWeight',params,0,deployer_wallet)

# 2:00
# params = {
#     'type': 'reserve',
#     'weights': [
#         {
#             'address': contracts['oICX'],
#             'weight': 5 * 10 ** 16          
#         },{
#             'address': contracts['oUSDS'],
#             'weight': 10 * 10 ** 16          
#         },{
#             'address': contracts['dIUSDC'],
#             'weight': 80 * 10 ** 16          
#         }
#     ],
#     'timestamp': 1653639300 * 10 ** 6
# }

# send_tx('rewardWeightController','setAssetWeight',params,0,deployer_wallet)


# # 3:00
# params = {
#     'type': 'reserve',
#     'weights': [
#         {
#             'address': contracts['oICX'],
#             'weight': 40 * 10 ** 16          
#         },{
#             'address': contracts['oUSDS'],
#             'weight': 20 * 10 ** 16          
#         },{
#             'address': contracts['dIUSDC'],
#             'weight': 35 * 10 ** 16          
#         }
#     ],
#     'timestamp': 1653642900 * 10 ** 6
# }

# send_tx('rewardWeightController','setAssetWeight',params,0,deployer_wallet)


Calling setAssetWeight, with parameters {'type': 'reserve', 'weights': [{'address': 'cx400962267deaaa5cc528297657d4c2d1452f9902', 'weight': 400000000000000000}, {'address': 'cx1b0a15ac99760720d2cdab4d2309801900ddaece', 'weight': 200000000000000000}, {'address': 'cx93239368e19d2028b2abca0472772157d44f2b74', 'weight': 350000000000000000}], 'timestamp': 1653642900000000} on the rewardWeightController contract.
rewardWeightController ::> setAssetWeight ::> 0xd751003bc4639af3913d5ce7ae57f1bfe182aa224745bd374edc66002a4795cf
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 2 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 4 seconds...
SCORE rewardWeightController ::: Method setAssetWeight ::: Status: 1


'0xd751003bc4639af3913d5ce7ae57f1bfe182aa224745bd374edc66002a4795cf'

In [ ]:
"""
12:30
# typeWeight
{'key': 'workerToken', 'weight': 30 * 10 ** 16},
{'key': 'daoFund', 'weight': 40 * 10 ** 16},
{'key': 'OMMLocking', 'weight': 5 * 10 ** 16},
{'key': 'reserve', 'weight': 10 * 10 ** 16},
{'key': 'liquidity', 'weight': 15 * 10 ** 16}

1:30
# typeWeight
{'key': 'workerToken', 'weight': 10 * 10 ** 16},
{'key': 'daoFund', 'weight': 10 * 10 ** 16},
{'key': 'OMMLocking', 'weight': 40 * 10 ** 16},
{'key': 'reserve', 'weight': 15 * 10 ** 16},
{'key': 'liquidity', 'weight': 25 * 10 ** 16}

2:00
# typeWeight
{'key': 'workerToken', 'weight': 20 * 10 ** 16},
{'key': 'daoFund', 'weight': 15 * 10 ** 16},
{'key': 'OMMLocking', 'weight': 5 * 10 ** 16},
{'key': 'reserve', 'weight': 35 * 10 ** 16},
{'key': 'liquidity', 'weight': 25 * 10 ** 16}
"""

In [85]:
# 12:30
# params = {
#     "weights": [
#          {'key': 'workerToken', 'weight': 30 * 10 ** 16},
#          {'key': 'daoFund', 'weight': 40 * 10 ** 16},
#          {'key': 'OMMLocking', 'weight': 5 * 10 ** 16},
#          {'key': 'reserve', 'weight': 10 * 10 ** 16},
#          {'key': 'liquidity', 'weight': 15 * 10 ** 16}
#     ],    
#     "timestamp": 1653633900 * 10 ** 6
# }
# send_tx('rewardWeightController','setTypeWeight',params,0,deployer_wallet)

#  1:30
# params = {
#     "weights": [
#          {'key': 'workerToken', 'weight': 10 * 10 ** 16},
#          {'key': 'daoFund', 'weight': 10 * 10 ** 16},
#          {'key': 'OMMLocking', 'weight': 40 * 10 ** 16},
#          {'key': 'reserve', 'weight': 15 * 10 ** 16},
#          {'key': 'liquidity', 'weight': 25 * 10 ** 16}
#     ],    
#     "timestamp": 1653637500 * 10 ** 6
# }
# send_tx('rewardWeightController','setTypeWeight',params,0,deployer_wallet)

# 2:00
# params = {
#     "weights": [
#          {'key': 'workerToken', 'weight': 20 * 10 ** 16},
#          {'key': 'daoFund', 'weight': 15 * 10 ** 16},
#          {'key': 'OMMLocking', 'weight': 5 * 10 ** 16},
#          {'key': 'reserve', 'weight': 35 * 10 ** 16},
#          {'key': 'liquidity', 'weight': 25 * 10 ** 16}
#     ],    
#     "timestamp": 1653639300 * 10 ** 6
# }
# send_tx('rewardWeightController','setTypeWeight',params,0,deployer_wallet)

Calling setTypeWeight, with parameters {'weights': [{'key': 'workerToken', 'weight': 100000000000000000}, {'key': 'daoFund', 'weight': 100000000000000000}, {'key': 'OMMLocking', 'weight': 400000000000000000}, {'key': 'reserve', 'weight': 150000000000000000}, {'key': 'liquidity', 'weight': 250000000000000000}], 'timestamp': 1653637500000000} on the rewardWeightController contract.
rewardWeightController ::> setTypeWeight ::> 0x08fff229366ba2067f2b2e4f82bf03c1df7a15aeb0cae2b333938fe59dd97d09
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE rewardWeightController ::: Method setTypeWeight ::: Status: 1
Calling setTypeWeight, with parameters {'weights': [{'key': 'workerToken', 'weight': 200000000000000000}, {'key': 'daoFund', 'weight': 150000000000000000}, {'key': 'OMMLocking', 'weight': 50000000000000000}, {'key': 'reserve', 'weight': 350000000000000000}, {'key': 'liquidity', 'weigh

'0x55aaf7cc00a0b460867f353fb4aa03877f253319cccf66a5a335ea344d87e810'

In [99]:
# CLAIM REWARDS
#users perform transaction
claim = {}
for pk in private_keys:
    wallet = KeyWallet.load(bytes.fromhex(pk))
    claim[wallet.get_address()] = ts(send_tx('lendingPool','claimRewards',{},0,wallet))

Calling claimRewards, with parameters {} on the lendingPool contract.
lendingPool ::> claimRewards ::> 0x8481ddacb7a2ea0c6b4ffbb5993b553ad7e8e7d122ae1c848f9b71c239e06e47
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE lendingPool ::: Method claimRewards ::: Status: 1
Calling claimRewards, with parameters {} on the lendingPool contract.
lendingPool ::> claimRewards ::> 0x83d875a4d52280fb9038117e80965e89a8ef3471c599fcf368a691bf54b518c2
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE lendingPool ::: Method claimRewards ::: Status: 1
'NoneType' object has no attribute 'get', Retrying in 1 seconds...
Calling claimRewards, with parameters {} on the lendingPool contract.
lendingPool ::> claimRewards ::> 0x83e7e4b4b7b13d89e4ef01fe2d745c365a885370f71e5939914c173f5e380252
{'c

In [261]:
twelve_30 = t1 = 1653633900
one_30 = t2 = 1653637500
two_0 = t3 =1653639300
three_0 = t4 = 1653642900

In [157]:
e1 = hexToInt(call_tx('rewardWeightController','getEmissionRate',{'timestamp': t1 * 10 ** 6 })) # t1 - t0
e2 = hexToInt(call_tx('rewardWeightController','getEmissionRate',{'timestamp': t2 * 10 ** 6 })) # t2 - t1
e3 = hexToInt(call_tx('rewardWeightController','getEmissionRate',{'timestamp': t3 * 10 ** 6 })) # t3 - t2
e4 = hexToInt(call_tx('rewardWeightController','getEmissionRate',{'timestamp': t4 * 10 ** 6 })) # t4 - t3
e5 = hexToInt(call_tx('rewardWeightController','getEmissionRate',{'timestamp': t4 * 10 ** 6 + 1 })) # after t4

In [160]:
asset1 = 'oICX'
asset2 = 'oUSDS'
asset3 = 'dIUSDC'
asset4 = 'OMM/USDS'
asset5 = "bOMM"

In [245]:
"""
claim dekhi 12:30  -> e1
12:30 dekhi 1:30 -> e2
1:30 dekhi 2:00 -> e3
2:00 dekhi 3:00 -> e4
3:00 dekhi after that -> e5
"""

In [262]:
working_total = call_tx('rewardDistribution','getWorkingTotal',{})

for idx,addr in enumerate(addresses):
    reward = call_tx('rewardDistribution','getRewards',{'_user': addr})
    t5 = int(reward.get('now'),0)
    
    # expected rewards calculation
    claimed_at = t0 = claim.get(addr)     
    
    working_balances = call_tx('rewardDistribution','getWorkingBalances',{'user': addr})
    
    ratio1 = int(working_balances.get(asset1),0)/int(working_total.get(asset1),0)
    ratio2 = int(working_balances.get(asset2),0)/int(working_total.get(asset2),0)
    ratio3 = int(working_balances.get(asset3),0)/int(working_total.get(asset3),0)
    ratio4 = int(working_balances.get(asset4),0)/int(working_total.get(asset4),0)
    ratio5 = int(working_balances.get(asset5),0)/int(working_total.get(asset5),0)
    
    cx = contracts[asset1]
    expected1 = (
        (t1 - t0) * e1.get(cx) + 
        (t2 - t1) * e2.get(cx) + 
        (t3 - t2) * e3.get(cx) + 
        (t4 - t3) * e4.get(cx) +
        (t5 - t4) * e5.get(cx)
    ) * ratio1 /10**18
    
    cx = contracts[asset2]
    expected2 = (
        (t1 - t0) * e1.get(cx) + 
        (t2 - t1) * e2.get(cx) + 
        (t3 - t2) * e3.get(cx) + 
        (t4 - t3) * e4.get(cx) +
        (t5 - t4) * e5.get(cx)
        ) * ratio2/10**18
    
    cx = contracts[asset3]
    expected3 = (
        (t1 - t0) * e1.get(cx) + 
        (t2 - t1) * e2.get(cx) + 
        (t3 - t2) * e3.get(cx) + 
        (t4 - t3) * e4.get(cx) +
        (t5 - t4) * e5.get(cx)
        ) * ratio3/10**18
    
    cx = contracts[asset4]
    expected4 = (
        (t1 - t0) * e1.get(cx) + 
        (t2 - t1) * e2.get(cx) + 
        (t3 - t2) * e3.get(cx) + 
        (t4 - t3) * e4.get(cx) +
        (t5 - t4) * e5.get(cx)
        ) * ratio4/10**18
    
    cx = contracts[asset5]
    expected5 = (
        (t1 - t0) * e1.get(cx) + 
        (t2 - t1) * e2.get(cx) + 
        (t3 - t2) * e3.get(cx) + 
        (t4 - t3) * e4.get(cx) +
        (t5 - t4) * e5.get(cx)
        ) * ratio5/10**18
    
    
    reward1 = int(reward.get('reserve').get(asset1),0)/10**18
    reward2 = int(reward.get('reserve').get(asset2),0)/10**18
    reward3 = int(reward.get('reserve').get(asset3),0)/10**18
    reward4 = int(reward.get('liquidity').get(asset4),0)/10**18
    reward5 = int(reward.get('OMMLocking').get(asset5),0)/10**18
    
    print(f"{idx} ::: {addr}")
    print(f"{asset1} \nexpected:> {expected1} actual:> {reward1} difference :> {abs(reward1-expected1)}")
    print(f"{asset2} \nexpected:> {expected2} actual:> {reward2} difference :> {abs(reward2-expected2)}")
    print(f"{asset3} \nexpected:> {expected3} actual:> {reward3} difference :> {abs(reward3-expected3)}")
    print(f"{asset4} \nexpected:> {expected4} actual:> {reward4} difference :> {abs(reward4-expected4)}")
    print(f"{asset5} \nexpected:> {expected5} actual:> {reward5} difference :> {abs(reward5-expected5)}")
    print("\n")

0 ::: hx687633a4c5870347e22fe0e3e07205b3b04485ad
oICX 
expected:> 2.3784606870798957 actual:> 2.3785377837865065 difference :> 7.709670661082413e-05
oUSDS 
expected:> 43.613004252838174 actual:> 43.61540952809641 difference :> 0.00240527525823353
dIUSDC 
expected:> 205.67352363695213 actual:> 205.67961934926785 difference :> 0.006095712315726587
OMM/USDS 
expected:> 3.0700195109976853 actual:> 3.070217901022952 difference :> 0.00019839002526644833
bOMM 
expected:> 106.85414186724465 actual:> 106.85866320283377 difference :> 0.004521335589117825


1 ::: hx948679df839205f7e4c7bf261b92258c04f4653c
oICX 
expected:> 11.895331648330128 actual:> 11.895345759828421 difference :> 1.411149829344538e-05
oUSDS 
expected:> 43.6277706834023 actual:> 43.62878210312345 difference :> 0.0010114197211521514
dIUSDC 
expected:> 205.7015965682195 actual:> 205.70425913446346 difference :> 0.002662566243969877
OMM/USDS 
expected:> 3.0692920809050395 actual:> 3.0692920809050395 difference :> 0.0
bOMM 
expected

In [264]:
ratios = [[asset1, asset2, asset3, asset4, asset5]]
for idx,addr in enumerate(addresses):
    reward = call_tx('rewardDistribution','getRewards',{'_user': addr})
    t5 = int(reward.get('now'),0)
    
    # expected rewards calculation
    claimed_at = t0 = claim.get(addr)     
    
    working_balances = call_tx('rewardDistribution','getWorkingBalances',{'user': addr})
    
    ratio1 = int(working_balances.get(asset1),0)/int(working_total.get(asset1),0)
    ratio2 = int(working_balances.get(asset2),0)/int(working_total.get(asset2),0)
    ratio3 = int(working_balances.get(asset3),0)/int(working_total.get(asset3),0)
    ratio4 = int(working_balances.get(asset4),0)/int(working_total.get(asset4),0)
    ratio5 = int(working_balances.get(asset5),0)/int(working_total.get(asset5),0)
    ratios.append([ratio1, ratio2, ratio3, ratio4, ratio5])
    
ratios

[['oICX', 'oUSDS', 'dIUSDC', 'OMM/USDS', 'bOMM'],
 [0.00084947271820891,
  0.030376657160496083,
  0.03465001802138775,
  0.0008570449091532396,
  0.019533255453050815],
 [0.00424736359104455,
  0.030376657160496083,
  0.03465001802138775,
  0.0008570449091532396,
  0.019533250465142052],
 [0.00424736359104455,
  0.060753314320992166,
  0.0693000360427755,
  0.0017140898183064792,
  0.07827006148069095],
 [0.00169894543641782,
  0.09112997148148824,
  0.10395005406416324,
  0.002571134727459719,
  0.17621039037659653]]

In [267]:
beforeIndexes = defaultdict(dict)
for addr in addresses:
    for asset in [asset1, asset2, asset3, asset4, asset5]:
        index = hexToInt(call_tx('rewardDistribution','getIndexes',{'_user': addr, '_asset': contracts[asset]}))
        beforeIndexes[addr][asset] = index
beforeIndexes

defaultdict(dict,
            {'hx687633a4c5870347e22fe0e3e07205b3b04485ad': {'oICX': {'assetIndex': 9389535678352694675,
               'userIndex': 9387618057423297719},
              'oUSDS': {'assetIndex': 3242677882991824583,
               'userIndex': 3242471007765591094},
              'dIUSDC': {'assetIndex': 82106844381018059492,
               'userIndex': 82104454177650985279},
              'OMM/USDS': {'assetIndex': 17723779551187692955770,
               'userIndex': 17718423020505485208095},
              'bOMM': {'assetIndex': 978396493244026946849,
               'userIndex': 978394036295163199760}},
             'hx948679df839205f7e4c7bf261b92258c04f4653c': {'oICX': {'assetIndex': 9389535678352694675,
               'userIndex': 9388187857582539094},
              'oUSDS': {'assetIndex': 3242677882991824583,
               'userIndex': 3242534167756175726},
              'dIUSDC': {'assetIndex': 82106844381018059492,
               'userIndex': 82105187293579918886},

In [268]:
# users perform transaction again
finalClaim={}
for pk in private_keys:
    wallet = KeyWallet.load(bytes.fromhex(pk))
    finalClaim[wallet.get_address()] = ts(send_tx('lendingPool','claimRewards',{},0,wallet))

Calling claimRewards, with parameters {} on the lendingPool contract.
lendingPool ::> claimRewards ::> 0xdd48c3794a0b9000aa63c3887ceb085e4d4c3c36377bd8f7fa2afd598656ec1c
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE lendingPool ::: Method claimRewards ::: Status: 1
Calling claimRewards, with parameters {} on the lendingPool contract.
lendingPool ::> claimRewards ::> 0x90e790fb564dade6c4c246c016ed335e6509a05db58733a83caee2f9f6c93dbc
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 1 seconds...
SCORE lendingPool ::: Method claimRewards ::: Status: 1
'NoneType' object has no attribute 'get', Retrying in 1 seconds...
'NoneType' object has no attribute 'get', Retrying in 2 seconds...
Calling claimRewards, with parameters {} on the lendingPool contract.
lendingPool ::> claimRewards ::> 0xc8d25e50950d65dbb0c483088a7235780bdd88faf1be2cc7ec72f69583cb1473
{'code': -3100

In [304]:
afterIndexes = defaultdict(dict)
for addr in addresses:
    for asset in [asset1, asset2, asset3, asset4, asset5]:
        index = hexToInt(call_tx('rewardDistribution','getIndexes',{'_user': addr, '_asset': contracts[asset]}))
        afterIndexes[addr][asset] = index
afterIndexes

defaultdict(dict,
            {'hx687633a4c5870347e22fe0e3e07205b3b04485ad': {'oICX': {'assetIndex': 12659753947836789777,
               'userIndex': 12645438759437343323},
              'oUSDS': {'assetIndex': 3838397881750442206,
               'userIndex': 3835838367119326332},
              'dIUSDC': {'assetIndex': 93983118005754947704,
               'userIndex': 93957571638301679190},
              'OMM/USDS': {'assetIndex': 21325285939875546616298,
               'userIndex': 21316689038780645292855},
              'bOMM': {'assetIndex': 980692289235365191707,
               'userIndex': 980689923232459912872}},
             'hx948679df839205f7e4c7bf261b92258c04f4653c': {'oICX': {'assetIndex': 12659753947836789777,
               'userIndex': 12648742264452600197},
              'oUSDS': {'assetIndex': 3838397881750442206,
               'userIndex': 3836429024341891534},
              'dIUSDC': {'assetIndex': 93983118005754947704,
               'userIndex': 939634669538678180

In [317]:
expectedIndex = {}
assetList = [asset1, asset2, asset3, asset4, asset5]
deltaTime = finalClaim.get(addr) - claim.get(addr)
for asset in assetList:        
    total = hexToInt(call_tx('rewardDistribution','getWorkingTotal',{})).get(asset)

    emissionRate = e5.get(contracts.get(asset))
    beforeAssetIndex = beforeIndexes.get(addr).get(asset).get('assetIndex')
    expectedIndex[asset] = int(beforeAssetIndex + 
                               (((t1 - claim.get(addr)) * e1.get(contracts.get(asset))+
                               (t2 - t1) * e2.get(contracts.get(asset))+
                               (t3 - t2) * e3.get(contracts.get(asset))+
                               (t4 - t3) * e4.get(contracts.get(asset))+
                               (finalClaim.get(addr) - t4) * e5.get(contracts.get(asset))) * 10 ** 18 )                              
                               / total)
    

{'oICX': 12660304532006000640,
 'oUSDS': 3838496324620869632,
 'dIUSDC': 93984100558349303808,
 'OMM/USDS': 21325616589917659332608,
 'bOMM': 980692506698037723136}

In [321]:
for asset in assetList:
    print(f"Expected {asset} :> {expectedIndex[asset]/10**18} Actual {asset}: {afterIndexes.get(addr).get(asset).get('assetIndex')/10**18}")

Expected oICX :> 12.660304532006 Actual oICX: 12.65975394783679
Expected oUSDS :> 3.8384963246208694 Actual oUSDS: 3.838397881750442
Expected dIUSDC :> 93.9841005583493 Actual dIUSDC: 93.98311800575495
Expected OMM/USDS :> 21325.61658991766 Actual OMM/USDS: 21325.285939875546
Expected bOMM :> 980.6925066980377 Actual bOMM: 980.6922892353651


## DIstribute

In [322]:
send_tx('rewardDistribution','distribute',{},0,deployer_wallet)

Calling distribute, with parameters {} on the rewardDistribution contract.
rewardDistribution ::> distribute ::> 0xa9f1759d257299b4ca02d583feeb45e81572f7dbd7eaf8886750000a6af0166d
{'code': -31002, 'message': 'Pending: Pending'}, Retrying in 1 seconds...
{'code': -31003, 'message': 'Executing: Executing'}, Retrying in 2 seconds...
SCORE rewardDistribution ::: Method distribute ::: Status: 1


'0xa9f1759d257299b4ca02d583feeb45e81572f7dbd7eaf8886750000a6af0166d'